# Fairness in Social Influence Maximization via Optimal Transport  
### Authors: Guillaume MARIN-BERTIN & Jaishan BURTON ELMO  

**Table of Contents**  
- 1. [Introduction](#introduction)  
- 2. [Related Work](#related-work)  
- 3. [Fairness via Optimal Transport](#fairness-via-optimal-transport)  
- 4. [Improving Fairness](#improving-fairness)  
- 5. [Evaluation](#evaluation)  
- 6. [Results](#results)  
- 7. [Conclusion](#conclusion)  
- [Appendix](#appendix)  
- [References](#references)  

This is a blog post about the article “Fairness in Social Influence Maximization via Optimal Transport” published by Shubham Chowdhary et al. in 2024 and available [**here**](https://neurips.cc/virtual/2024/poster/94521).

### [1. Introduction](#introduction)  
Social networks have become a crucial vector for spreading information, influencing public opinion, and promoting products or ideas. In this context, the **Influence Maximization (IM)** problem aims to identify the most strategic individuals to initiate a diffusion process and maximize its reach. Applications of IM range from viral marketing to public health campaigns and disinformation detection.  

However, traditional IM algorithms focus solely on maximizing the number of influenced individuals without considering fairness. This can lead to unequal information diffusion, where certain groups are systematically disadvantaged. Addressing this issue is crucial, particularly in sensitive applications such as political campaigns, healthcare awareness, and social justice movements.  

In this blog post, we introduce a new fairness-aware approach to Influence Maximization based on **Optimal Transport (OT)**. Unlike existing methods, our approach ensures that the information spreads in an equitable manner across different demographic groups.

### [2. Related Work](#related-work)

The Social Influence Maximization (SIM) problem has been widely studied, with research efforts primarily focused on optimizing information diffusion in social networks. Existing approaches can be broadly classified into three categories: greedy-based heuristics, fairness-aware methods, and optimal transport-based approaches. 

**Greedy-based heuristics**
Early works, such as Kempe et al. (2003), formulated SIM as a submodular optimization problem, showing that a greedy strategy achieves a \((1 - 1/e)\)-approximation guarantee. Later methods leveraged heuristics like degree centrality, PageRank, and community-based seed selection to improve efficiency. However, these approaches do not account for fairness, potentially reinforcing structural biases in social networks.

**Fairness-aware methods**  
To address fairness concerns, recent studies introduced constraints ensuring more equitable information diffusion across demographic groups. Several fairness criteria have been explored: 

- Equity-based fairness (*Stoica et al., 2020*) ensures that, in expectation, the same proportion of users in each group is reached.  
- Max-min fairness (*Fish et al., 2019; Zhu et al., 2019*) maximizes the minimum probability of a group receiving information to prevent exclusion.  
- Welfare-based approaches (*Rahmattalabi et al., 2021*) optimize outreach under social welfare constraints, while diversity-aware methods (*Tsang et al., 2019*) seek to prevent disproportionate advantages for certain groups.  

However, these approaches rely on marginal outreach probabilities, which fail to capture stochastic variations in diffusion. As a result, they can misclassify unfair scenarios as fair, despite significant disparities in influence spread.  

**Fairness via Optimal Transport**  
To overcome these limitations, we propose a fairness-aware framework based on Optimal Transport (OT). This method introduces a new Mutual Fairness metric and the Stochastic Seed Selection Descent (S3D) algorithm, ensuring a balanced trade-off between fairness and efficiency. Unlike prior methods, our approach explicitly accounts for joint probability distributions, providing a more robust measure of fairness in influence propagation.

### [3. Fairness via Optimal Transport](#fairness-via-optimal-transport)



### [4. Improving Fairness](#improving-fairness)

While Mutual Fairness provides a rigorous metric for evaluating fairness in influence maximization, ensuring that an influence propagation algorithm adheres to this fairness constraint requires a dedicated optimization strategy. To address this challenge, we introduce **Stochastic Seed Selection Descent (S3D)**, a new method designed to balance fairness and efficiency in social influence diffusion.

#### Stochastic Seed Selection Descent (S3D)

S3D is an iterative fairness-aware seed selection algorithm that optimizes seed placement while ensuring a fair distribution of influence. Unlike traditional influence maximization strategies that prioritize outreach without considering fairness constraints, S3D actively adjusts the selection process to minimize unfairness as measured by the Mutual Fairness metric.

The algorithm follows these key steps:

1. **Initial Seed Selection**:  
   The process begins with an initial set of influential nodes chosen based on traditional heuristics (e.g., degree centrality or community detection).

2. **Exploration of Neighboring States**:  
   At each iteration, the algorithm samples alternative seed sets by adding, swapping, or removing nodes from the current selection.   

3. **Fairness Evaluation**:  
   Each candidate seed set is evaluated based on the β-Fairness metric, which balances fairness and efficiency. 
   The fairness-energy function E(S) is computed for the current and candidate seed sets to assess the fairness gain or loss.

4. **Acceptance Criteria (Metropolis-Hastings Selection Rule)**:  
   The new seed set \(S'\) is accepted with a probability defined as:
$$
p_{\text{accept}} \leftarrow \min \left\{ 1, e^{E_S - E_{S'}} \right\}
$$

   where \(E(S)\) represents the fairness-energy function of the current seed set. This criterion ensures that modifications leading to improved fairness are more likely to be accepted, while allowing some degree of randomness to explore the solution space.

5. **Convergence**:  
   The process continues until the fairness score stabilizes, ensuring that the final seed set achieves an optimal balance between outreach and fairness.

#### Impact of S3D on Influence Spread

S3D significantly improves fairness in social influence diffusion by explicitly considering joint probability distributions of outreach, rather than relying solely on marginal probabilities. This results in:

- **More equitable information diffusion**: Minority groups are no longer systematically disadvantaged in the diffusion process.
- **Minimal trade-off with efficiency**: While fairness constraints often reduce total outreach, our experiments show that S3D maintains a competitive influence spread compared to traditional heuristics.
- **Scalability**: Despite its iterative nature, S3D remains computationally feasible for large-scale social networks, thanks to its efficient exploration strategy.

By introducing S3D, we provide a practical and effective solution for ensuring fairness in influence maximization, offering a significant improvement over traditional strategies while keeping computational complexity under control.